In [2]:
import sys
sys.path.append("/home/wgar/PRockClassify/")

In [3]:
import os
import argparse
import torch
import torch.optim
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

from custom_logging import get_logger
from data.DatasetManifest import DatasetManifest
from data.dataloader import MusicDataLoader, MusicDataset
from data.WindowedMFCCFeatureExtractor import WindowedMFCCFeatureExtractor
# from data.MFCCFeatureExtractor import MFCCFeatureExtractor
from EncoderDecoder.model import ConvED

In [4]:
opt = {"root_prog": "/media/wgar/New Volume1/dataset/cap6610sp19_project/Progressive Rock Songs/",
       "root_nonprog": "/media/wgar/New Volume1/dataset/cap6610sp19_project/0_Not_prog/",
       "num_workers": 4,
       "batch_size": 2,
       "seed": 9,
       "cuda": True,
       "debug_dataset_size": 32,
       "sample_rate": 44100
      }

In [5]:
torch.manual_seed(opt['seed'])
torch.cuda.manual_seed_all(opt['seed'])
np.random.seed(opt['seed'])

device = torch.device("cuda" if opt['cuda'] else 'cpu')

In [6]:
mfcc_feature_extractor = WindowedMFCCFeatureExtractor(opt["sample_rate"], 20, 512, 2048, 12)

In [7]:
dataset_manifest = DatasetManifest(opt, debug=True, quiet=False)

  DEBUG    | DatasetManifest | number of train songs:	16
  DEBUG    | DatasetManifest | number of val songs:	8
  DEBUG    | DatasetManifest | number of test songs:	8


In [8]:
mel_train = MusicDataset(opt, 'train', mfcc_feature_extractor, dataset_manifest,
                              quiet=True, pre_cache=True)
train_loader = MusicDataLoader(mel_train,
                               batch_size=opt["batch_size"],
                               shuffle=True,
                               num_workers=opt['num_workers'])

  INFO     | dataloader | Pre-caching features...
100%|██████████| 16/16 [00:17<00:00,  1.06s/it]
  INFO     | dataloader | Finished initializing dataloader.


In [9]:
mt0 = mel_train.__getitem__(0)['x']

In [10]:
mt0.shape

torch.Size([230, 48])

In [13]:
for data in train_loader:
    print(data[0].shape)
    print(data[1])
    print(data[2])

torch.Size([2, 1, 230, 48])
tensor([1, 0], dtype=torch.int32)
tensor([1., 1.])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1., 1.])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1., 1.])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1.0000, 0.7500])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1.0000, 0.7500])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1.0000, 0.7500])
torch.Size([2, 1, 230, 48])
tensor([0, 1], dtype=torch.int32)
tensor([1., 1.])
torch.Size([2, 1, 230, 48])
tensor([0, 0], dtype=torch.int32)
tensor([1.0000, 0.7500])


In [14]:
from EncoderDecoder.model import EncoderRNN, DecoderRNN, MaskConv

In [94]:
for data in train_loader:
    inputs, targets, input_percentages = data
    break

In [95]:
input_size = 48
hidden_size = 512
latent_size = 100

In [96]:
import torch.nn as nn

In [97]:
real_lengths = (inputs.shape[3] * input_percentages).int()
real_lengths

tensor([48, 36], dtype=torch.int32)

In [98]:
inputs_bak = inputs

In [99]:
print(inputs.shape)
print(inputs.squeeze(1).shape)
inputs.squeeze(1).transpose(1, 2).shape

torch.Size([2, 1, 230, 48])
torch.Size([2, 230, 48])


torch.Size([2, 48, 230])

In [100]:
inputs = inputs.squeeze(1).transpose(1, 2)

In [101]:
packed = nn.utils.rnn.pack_padded_sequence(inputs, real_lengths, batch_first=True)
packed.data.shape

torch.Size([84, 230])

In [102]:
unpacked, real_lengths = nn.utils.rnn.pad_packed_sequence(packed, batch_first=True)
unpacked.shape

torch.Size([2, 48, 230])

In [103]:
packed = nn.utils.rnn.pack_padded_sequence(inputs, real_lengths, batch_first=True)

In [111]:
rnn = nn.LSTM(input_size=230, hidden_size=100, bidirectional=False)

In [112]:
x, h = rnn(packed)

In [113]:
unpacked, real_lengths = nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

In [114]:
unpacked.shape

torch.Size([2, 48, 100])

In [115]:
fc = nn.Linear(100, 2)

In [116]:
wx = fc(unpacked)

In [118]:
wx.shape

torch.Size([2, 48, 2])